# <a id=NotebookOrg> Notebook Organization </a> 

## 1. [City Overview - Data Collection](#CityCountyLevel)

## 2. [Home Level - Data Collection](#HomeLevel)

## 3. [City Level - Data Collection](#CityLevel)


In [12]:
# All Imports
import numpy as np
import pandas as pd
import requests
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
from urllib.request import urlopen,Request
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.remote import webelement
import pandas as pd
import time
# Selenium
# Make sure to !pip install selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys


# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# <a id=CityCountyLevel>1. City Overview - Data Collection </a>   
[Back to Top](#NotebookOrg)

### In this section we access various pages within this page: to get the total homes available for rent in each city
### Main Page used: 'https://www.redfin.com/rental-sitemap/CA'
#### This section utilizes two different methods - 
#### (One) using beautiful soup and (second) using Selenium 


## Step 1: Selenium

In [13]:
driver = webdriver.Chrome('chromedriver.exe')
#driver.get('https://www.redfin.com/city/17519/CA/San-Ramon/housing-market#agent-insights')

#search_box = driver.find_element('searchInputBox')
# search_box.send_keys('687 Catalina Laguna Beach, CA 92651')
# search_box.submit()
# time.sleep(3)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# data = soup.find_all(lambda tag: tag.name == 'span' and tag.get('class') == ['value'])
# print(data)

driver.get('https://www.redfin.com/rental-sitemap/CA')
elementcss= driver.find_element(By.XPATH,"//ul[@class='list ldpsSection']")

<ipython-input-13-b1e13c2f2238>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [14]:
# Check how long it takes
start = time.time()


final_df_cities = pd.DataFrame( columns =['City Name ','Total Homes for Rent','City Link'])

text = "homes for rent in California"
cities = False
city_web_elements = []
elementListLength =len(driver.find_elements(By.TAG_NAME,"a"))
wait = WebDriverWait(driver, 1)

pattern = re.compile(r"")
for x in range(elementListLength):
    elementsList = driver.find_elements(By.TAG_NAME,"a")
    if cities == False:
        if elementsList[x].text == text:
            cities = True
    else:
        if "County" in elementsList[x].text: 
            break
        else:
            city_name = elementsList[x].text
            city_href = elementsList[x].get_attribute('href')
            elementsList[x].click()
            element = driver.find_element(By.XPATH,"//div[@class='homes summary']")
            city_homes = int((re.findall('\d+',element.text)[-1]))
            # Insert into dataframe
            final_df_cities.loc[x] = [city_name,city_homes,city_href]
            driver.back()
            
print(final_df_cities) 

end = time.time()
print('Processing Time: ',end - start)

          City Name  Total Homes for Rent  \
29            Corona                   38   
30            Dublin                   19   
31           Fremont                   84   
32         Fullerton                   24   
33            Irvine                  106   
34         Livermore                   14   
35        Long Beach                  362   
36       Los Angeles                 2272   
37           Oakland                  515   
38      Palm Springs                   90   
39          Pasadena                   81   
40        Pleasanton                   18   
41  Rancho Cucamonga                   30   
42         Riverside                   80   
43        Sacramento                  411   
44         San Diego                  663   
45     San Francisco                  899   
46          San Jose                  335   
47         San Ramon                   22   
48          Temecula                   29   

                                            City Link 

## Step 2: Beautiful Soup

In [15]:
# Check how long it takes
start_bs = time.time()

webpage = urlopen(Request('https://www.redfin.com/rental-sitemap/CA', headers={'User-Agent': 'Mozilla/5.0'}))
soup = BS(webpage,'html.parser')
    
final_df_cities_bs = pd.DataFrame( columns =['City Name ','Total Homes for Rent','City Link'])

text = "homes for rent in California"
cities = False
counter = 0 
for x in soup.find_all('a'):
    if cities == False:
        if x.get_text() == text:
             cities = True
    else:
        if "County" in x.get_text(): 
            break
        else:
            
            city_name = x.get_text()
            city_href = 'https://www.redfin.com'+x['href']
            webpage2 = urlopen(Request(city_href, headers={'User-Agent': 'Mozilla/5.0'}))
            soup2 = BS(webpage2,'html.parser')
            total_homes = (int((re.findall('\d+',soup2.find_all("div", {"class": "homes summary"})[0].get_text())[-1])))
#             wait.until(EC.element_to_be_clickable(elementsList[x])).click()
#             element = driver.find_element(By.XPATH,"//div[@class='homes summary']")
#             city_homes = int((re.findall('\d+',element.text)[-1]))
            # Insert into dataframe
            final_df_cities_bs.loc[counter] = [city_name,total_homes,city_href]  
            counter+=1
print(final_df_cities_bs)
end_bs = time.time()
print('Processing Time: ',end_bs - start_bs)

          City Name  Total Homes for Rent  \
0             Corona                   38   
1             Dublin                   19   
2            Fremont                   84   
3          Fullerton                   24   
4             Irvine                  106   
5          Livermore                   14   
6         Long Beach                  362   
7        Los Angeles                 2272   
8            Oakland                  515   
9       Palm Springs                   90   
10          Pasadena                   81   
11        Pleasanton                   18   
12  Rancho Cucamonga                   30   
13         Riverside                   80   
14        Sacramento                  411   
15         San Diego                  663   
16     San Francisco                  899   
17          San Jose                  335   
18         San Ramon                   22   
19          Temecula                   29   

                                            City Link 

## Step 3: Comparing Performance of Beautiful Soup vs Selenium

In [16]:
print('Beautiful Soup')
print('Time Taken: ',end_bs-start_bs)
print('Total Requests to website: ', len(final_df_cities_bs))

print('Selenium')
print('Time Taken: ',end-start)
print('Total Requests to website: ',1)


Beautiful Soup
Time Taken:  29.69718098640442
Total Requests to website:  20
Selenium
Time Taken:  55.97927284240723
Total Requests to website:  1


## Step 4: Export to Excel

In [17]:
#Read the dataframe in to an excel file
result=final_df_cities
writer=pd.ExcelWriter('Cities_Summary.xlsx') 
result.to_excel(writer,'city summary')
writer.save() 

# <a id=HomeLevel>2. Home Level - Data Collection </a> 
[Back to Top](#NotebookOrg)

#### This section uses the BeautifulSoup package to scrape all home data 6 of the top most populated cities in the Bay Area
#### Cupertino, Gilroy, Sunnyvale, Santa Clara, San Jose, Morgan Hill

### Step 1: Iteration
#### Loop through each of the pages in each city to extract all the house data

In [2]:
with requests.Session() as s:
    pages = np.arange(1, 3, 1)
    Bed_Bath_Sqft_df = []
    Price_df = []
    Address_df = []
    URL_df = []
    Walk_Score_df = []
    Transit_Score_df = []
    Bike_Score_df = []
    Places_Nearby_df =[]
    Status = []
    Property_Type = []
    HOA = []
    Sold_Year = []
    Community = []
    Lot_Size = []
    Redfin_Est = []
    Price_Per_Sqft = []
    Buyers_Agent_Commision = []
    Year_Built = []
    Time_On_Redfin = []
    Storm_Risk = []
    Drought_Risk = []
    Heat_Risk = []
    Fire_Risk = []
    city_df =[]
    domain_var = "https://www.redfin.com/city/"
    property_type_var = "filter/property-type=house+condo+townhouse+multifamily/"
    City_List=['4561/CA/Cupertino','7521/CA/Gilroy/','19457/CA/Sunnyvale/','19457/CA/Santa-Clara','17420/CA/San-Jose','12625/CA/Morgan-Hill']
    property_status_var = ['include=sold-3yr/','status=active/']
    
    for city in City_List:  # Loop to read data from the above mentioned Bay Area cities
        
        for status_var in property_status_var:    # Loop to read data for Sold and Active properties
            
            actual_status = ''
            if(status_var == 'include=sold-3yr/'):
                actual_status = 'Sold'
            elif(status_var == 'status=active/'):
                actual_status = 'Active'
                               
            for page in pages:   # Loop to read multiple pages of the property data
                
                url = domain_var + city + property_type_var + status_var + 'page-'+str(page)
                r = s.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
                soup = BS(r.content,'html.parser')    # main soup object to read the high level data of the properties
                
                my_table = soup.find_all(class_= ['HomeStatsV2'])
                for tag in my_table:
                    Bed_Bath_Sqft_df.append(tag.get_text())
                my_table = soup.find_all(class_= ['homecardV2Price'])
                for tag in my_table:
                    # changed by moumita
                    Price_df.append(tag.get_text().strip(': $').replace(",",""))
                my_table = soup.find_all(class_= ['link-and-anchor'])
                for tag in my_table:
                    Address_df.append(tag.get_text())
                my_url = soup.find_all('a',attrs = {'class':'slider-item'})
                for urllist in my_url:
                    URL_df.append('https://www.redfin.com/'+urllist.get('href'))
                    Status.append(actual_status)
                    city_string= re.search(r'/CA/([^/]+)', city).group(1)
                    city_df.append(city_string)
                    

# Initializing the dataframe and reading data into it

df = pd.DataFrame( columns =['City','Bed Bath Sqft','Price','Address','URL','Walk Score','Transit Score','Bike Score','Places Nearby','Storm Risk','Drought Risk','Heat Risk','Fire Risk'])
df['City'] = pd.Series(city_df,dtype = 'string')
df['Bed Bath Sqft'] = pd.Series(Bed_Bath_Sqft_df,dtype = 'string')
df['Address'] = pd.Series(Address_df,dtype = 'string')
df['Price'] = pd.Series(Price_df,dtype = 'float')
df['URL'] = pd.Series(URL_df,dtype = 'string')
df['Status'] = pd.Series(Status,dtype = 'string')
df

,City,Bed Bath Sqft,Price,Address,URL,Walk Score,Transit Score,Bike Score,Places Nearby,Storm Risk,Drought Risk,Heat Risk,Fire Risk,Status
0,Cupertino,"4 Beds2.5 Baths2,270 Sq. Ft.",2580000.0,"10551 Glenview Ave, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/10551-Gle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sold
1,Cupertino,"3 Beds2 Baths1,196 Sq. Ft.",1830000.0,"18861 Barnhart Ave, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/18861-Bar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sold
2,Cupertino,"3 Beds2 Baths1,115 Sq. Ft.",2120000.0,"7652 W Hill Ln, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/7652-W-Hi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sold
3,Cupertino,"2 Beds1.5 Baths1,216 Sq. Ft.",1300000.0,"10367 Mary Ave, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/10367-Mar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sold
4,Cupertino,"3 Beds2 Baths1,688 Sq. Ft.",2780000.0,"10250 Lockwood Dr, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/10250-Loc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,Morgan-Hill,—Beds—Baths—Sq. Ft.,1649000.0,"10868 Dougherty Ave, MORGAN HILL, CA 95037",https://www.redfin.com//CA/Morgan-Hill/10868-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active
634,Morgan-Hill,—Beds—Baths—Sq. Ft.,1899000.0,"19900 Dougherty Ave, MORGAN HILL, CA 95037",https://www.redfin.com//CA/Morgan-Hill/19900-D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active
635,Morgan-Hill,—Beds—Baths—Sq. Ft.,240000.0,"0 Croy Rd, MORGAN HILL, CA 95037",https://www.redfin.com//CA/MORGAN-HILL/CROY-RD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active
636,Morgan-Hill,—Beds—Baths0.62 Acre (Lot),375000.0,"17585 Blue Jay Dr, MORGAN HILL, CA 95037",https://www.redfin.com//CA/Morgan-Hill/17585-B...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active


### Step 2: Append & Save 
#### Loop through each of the pages in each city to extract all the house data

In [5]:
for ind in df.index:    # Loop to read data from the individual properties fetched above
    url =(df['URL'][ind])
    
    r = s.get(url, headers ={'User-Agent': 'Mozilla/5.0'})
    
    soup_house = BS(r.content,'html.parser')
    
# WALK SCORE  
    try:
        for tag1 in soup_house.find( class_ =["transport-icon-and-percentage walkscore"] ).find( class_ =["percentage"] ).find_all( class_ =["value fair","value good","value poor"] ):
            walk_score = tag1.get_text()
            if (walk_score is None): 
                walk_score=0
    except:
        walk_score=0
    Walk_Score_df.append(walk_score)
    

# TRANSIT SCORE
    try:
        for tag1 in soup_house.find( class_ =["transport-icon-and-percentage transitscore"] ).find( class_ =["percentage"] ).find_all( class_ =["value fair","value good","value poor"] ):
            transit_score = tag1.get_text()
            if (transit_score is None): 
                transit_score=0
    except:
        transit_score=0
    Transit_Score_df.append(transit_score) 
       

# BIKE SCORE      
    try:
        for tag1 in soup_house.find( class_ =["transport-icon-and-percentage bikescore"] ).find( class_ =["percentage"] ).find_all( class_ =["value fair","value good","value poor"] ):
            bike_score = tag1.get_text()
            if (bike_score is None): 
                bike_score=0
    except:
        bike_score=0
    Bike_Score_df.append(bike_score)
    
    
# PROPERTY TYPE
    try:
        actual_property_type_value = ''
        for property_type_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(property_type_raw.span.text == 'Property Type'):
                property_type_value_span = property_type_raw.find('span', attrs = {'class': 'content text-right'})
                actual_property_type_value = property_type_value_span.text
    except:
        actual_property_type_value = "None"            

    Property_Type.append(actual_property_type_value)
    
    
# HOA
    try:
        actual_hoa_value = 0 
        for hoa_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(hoa_raw.span.text == 'HOA Dues'):
                hoa_raw_span = hoa_raw.find('span', attrs = {'class': 'content text-right'})
                actual_hoa_value = hoa_raw_span.text.strip('$')
    except:
        actual_hoa_value = 0        

    HOA.append(actual_hoa_value)
    
    
# YEAR BUILT
    try:
        actual_year_built_value = 9999
        for year_built_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(year_built_raw.span.text == 'Year Built'):
                year_built_raw_span = year_built_raw.find('span', attrs = {'class': 'content text-right'})
                actual_year_built_value = year_built_raw_span.text
    except:
        actual_year_built_value = 9999
        
    Year_Built.append(actual_year_built_value)
    
    
# COMMUNITY
    try:
        actual_community_value = '' 
        for community_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(community_raw.span.text == 'Community'):
                community_raw_span = community_raw.find('span', attrs = {'class': 'content text-right'})
                actual_community_value = community_raw_span.text
    except:
        actual_community_value = "None"
        
    Community.append(actual_community_value)
    
    
# LOT SIZE
    try:
        actual_lot_size_value = 0 
        for lot_size_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(lot_size_raw.span.text == 'Lot Size'):
                lot_size_span = lot_size_raw.find('span', attrs = {'class': 'content text-right'})
                actual_lot_size_value = lot_size_span.text.strip(' Sq.Ft.').strip(',')
    except:
        actual_lot_size_value = 0
        
    Lot_Size.append(actual_lot_size_value)
    
    
# TIME ON REDFIN
    try:
        actual_time_on_redfin_value = 0 
        for time_on_redfin_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(time_on_redfin_raw.span.text == 'Time on Redfin'):
                time_on_redfin_span = time_on_redfin_raw.find('span', attrs = {'class': 'content text-right'})
                actual_time_on_redfin_value = time_on_redfin_span.text 
    except:
        actual_time_on_redfin_value = 0
        
    Time_On_Redfin.append(actual_time_on_redfin_value)

    

    
# REDFIN ESTIMATE
    try:
        actual_redfin_est_value = 0
        for redfin_est_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base', 'data-rf-test-id':'key-detail-estimate'}):
            redfin_est_value_span = redfin_est_raw.find('span', attrs = {'class': 'content text-right'})
            actual_redfin_est_value = redfin_est_value_span.text.strip('$').strip(',')
    except:
        actual_redfin_est_value = 0
        
    Redfin_Est.append(actual_redfin_est_value)
    
    
    
# PRICE/SQ.FT.
    try:
        actual_price_per_sqft_value = 0
        for price_per_sqft_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
            if(price_per_sqft_raw.span.text == 'Price/Sq.Ft.'):
                price_per_sqft_span = price_per_sqft_raw.find('span', attrs = {'class': 'content text-right'})
                actual_price_per_sqft_value = price_per_sqft_span.text.strip('$').strip(',')
    except:
        actual_price_per_sqft_value = 0
        
    Price_Per_Sqft.append(actual_price_per_sqft_value)
    
    
# BUYER'S AGENT COMMISSION
    try:
        actual_buyers_agent_commission_value = 0
        for buyers_agent_commission_raw in soup_house.findAll('div', attrs = {'class':'keyDetail font-weight-roman font-size-base'}):
                buyers_agent_commission_span = buyers_agent_commission_raw.find('span', attrs = {'class': 'content text-right'})
                actual_buyers_agent_commission_value = buyers_agent_commission_span.text.strip('%')
    except:
        actual_buyers_agent_commission_value = 0
            
    Buyers_Agent_Commision.append(actual_buyers_agent_commission_value)
    
    
# CLIMATE ATTRIBUTES   
    try:
        Climate_List = []
        
        for attr in soup_house.find_all('b', attrs = {'class':'riskScoreLevel'}):
            Climate_List.append(attr.string)
            
    except:
        Climate_List = ["None", "None", "None", "None"]
   
    
    try:
        for risk in Climate_List:
            Storm_Risk.append(Climate_List[0])
            Drought_Risk.append(Climate_List[1])
            Heat_Risk.append(Climate_List[2])
            Fire_Risk.append(Climate_List[3])
    except:
        Storm_Risk.append("None")
        Drought_Risk.append("None")
        Heat_Risk.append("None")
        Fire_Risk.append("None")
    

In [6]:
# Assign data to the dataframe and display the content
df['Walk Score'] = Walk_Score_df
df['Transit Score'] = Transit_Score_df
df['Bike Score'] = Bike_Score_df

df['Property Type'] = pd.Series(Property_Type)
df['Hoa Dues'] = pd.Series(HOA)
df['Year Built'] =  pd.Series(Year_Built)
df['Community'] =  pd.Series(Community)
df['Lot Size'] =  pd.Series(Lot_Size)
df['Time on Redfin'] = pd.Series(Time_On_Redfin)

df['Redfin Estimate'] =  pd.Series(Redfin_Est)
df['Price Per Sq.Ft.'] =  pd.Series(Price_Per_Sqft)
df['Buyers Agent Commission'] =  pd.Series(Buyers_Agent_Commision)
df['Storm Risk'] = pd.Series(Storm_Risk)
df['Drought Risk'] = pd.Series(Drought_Risk)
df['Heat Risk'] = pd.Series(Heat_Risk)
df['Fire Risk'] = pd.Series(Fire_Risk)

df


,City,Bed Bath Sqft,Price,Address,URL,Walk Score,Transit Score,Bike Score,Places Nearby,Storm Risk,...,Status,Property Type,Hoa Dues,Year Built,Community,Lot Size,Time on Redfin,Redfin Estimate,Price Per Sq.Ft.,Buyers Agent Commission
0,Cupertino,"2 Beds2.5 Baths1,216 Sq. Ft.",1200000.0,"10357 Mary Ave, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/10357-Mar...,32,39,66,NaN,Moderate,...,Sold,Townhouse,398,1971,CUPERTINO,"1,600",0,"1,313,656","1,080",2.5
1,Cupertino,"6 Beds4.5 Baths3,730 Sq. Ft.",3530000.0,"10121 Camino Vista Dr, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/10121-Cam...,42,20,74,NaN,Moderate,...,Sold,Single Family Home,0,1999,CUPERTINO,"10,325",0,"4,653,543","1,248",2.5
2,Cupertino,"3 Beds2 Baths1,512 Sq. Ft.",1862000.0,"795 W Homestead Rd, SUNNYVALE, CA 94087",https://www.redfin.com//CA/Sunnyvale/795-W-Hom...,74,34,82,NaN,Moderate,...,Sold,Single Family Home,0,1960,SUNNYVALE,"8,102",0,"3,080,802","2,038",2.5
3,Cupertino,"2 Beds1.5 Baths1,216 Sq. Ft.",1300000.0,"10367 Mary Ave, CUPERTINO, CA 95014",https://www.redfin.com//CA/Cupertino/10367-Mar...,39,41,73,NaN,Moderate,...,Sold,Townhouse,398,1971,CUPERTINO,537,0,"1,357,211","1,069",2.5
4,Cupertino,"4 Beds2.5 Baths1,983 Sq. Ft.",2385000.0,"21614 La Playa Ct, Cupertino, CA 95014",https://www.redfin.com//CA/Cupertino/21614-La-...,9,0,22,NaN,High,...,Sold,Single Family Residence,0,1966,699 - Not Defined,0.25 Acres,0,"3,456,437","1,743",2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,Morgan-Hill,—Beds—Baths—Sq. Ft.,240000.0,"0 Croy Rd, MORGAN HILL, CA 95037",https://www.redfin.com//CA/MORGAN-HILL/CROY-RD...,0,0,0,NaN,Moderate,...,Active,Residential Development Land,0,9999,MORGAN HILL,13 Acres,74 days,0,0,3
594,Morgan-Hill,—Beds—Baths0.62 Acre (Lot),375000.0,"17585 Blue Jay Dr, MORGAN HILL, CA 95037",https://www.redfin.com//CA/Morgan-Hill/17585-B...,1,0,8,NaN,Moderate,...,Active,Residential Lot,73,9999,MORGAN HILL,0.62 Acres,88 days,"369,189",0,2.5
595,Morgan-Hill,—Beds—Baths2.01 Acres (Lot),296000.0,"14055 Prom Dr, MORGAN HILL, CA 95037",https://www.redfin.com//CA/Morgan-Hill/14055-P...,1,0,0,NaN,Moderate,...,Active,Residential Lot,0,9999,MORGAN HILL,2.01 Acres,89 days,"286,724",0,2.5
596,Morgan-Hill,—Beds—Baths5.01 Acres (Lot),350000.0,"15580 Armsby Ln, MORGAN HILL, CA 95037",https://www.redfin.com//CA/Morgan-Hill/15580-A...,0,0,0,NaN,Moderate,...,Active,Residential Lot,10,9999,MORGAN HILL,5.01 Acres,92 days,"327,670",0,2.5


### Step 3: Save result in excel file

In [7]:
# Read the dataframe in to an excel file
result=df
writer=pd.ExcelWriter('Project_HomeData.xlsx') 
result.to_excel(writer,'homes')
writer.save() 

# <a id=CityLevel>3. City Level - Data Collection </a> 
[Back to Top](#NotebookOrg)

#### This section uses the BeautifulSoup package to scrape all home data 13 chosen cities based on population, popularity in the Bay Area

#### Cupertino, Gilroy, Sunnyvale, Santa Clara, San Jose, Morgan Hill, Los Altos, Saratoga, Campbell, Los Gatos, Palo Alto, Milipitas, Mountain View

### Step 1: Iteration
#### Loop through each of the pages in each city to extract all the house data

In [11]:
from urllib.request import urlopen,Request
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
req1 = Request('https://www.redfin.com/city/19457/CA/Sunnyvale/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req2 = Request('https://www.redfin.com/city/17420/CA/San-Jose/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req3 = Request('https://www.redfin.com/city/4561/CA/Cupertino/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req4 = Request('https://www.redfin.com/city/12739/CA/Mountain-View/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req5 = Request('https://www.redfin.com/city/12204/CA/Milpitas/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req6 = Request('https://www.redfin.com/city/14325/CA/Palo-Alto/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req7 = Request('https://www.redfin.com/city/17675/CA/Santa-Clara/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req8 = Request('https://www.redfin.com/city/11234/CA/Los-Gatos/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req9 = Request('https://www.redfin.com/city/11234/CA/Campbell/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req10 = Request('https://www.redfin.com/city/7521/CA/Gilroy/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req11 = Request('https://www.redfin.com/city/17960/CA/Saratoga/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req12 = Request('https://www.redfin.com/city/12625/CA/Morgan-Hill/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
req13 = Request('https://www.redfin.com/city/11018/CA/Los-Altos/housing-market#trends', headers={'User-Agent': 'Mozilla/5.0'})
listreq = [req1,req2,req3,req4,req5,req6,req7,req8,req9,req10,req11,req12,req13]
stats= {"Competitive_Score":[],"#of houses sold":[],"Home sold above list price":[],"School rating":[]}
competitative = []
for r in listreq:
    webpage = urlopen(r)
    soup = BS(webpage,'html.parser')
    
# COMPETITIVE SCORE
    for i in soup.findAll('div',attrs={'class':"score most"})[:1]:
              c= i.get_text()  
    competitative.append(c)
    stats["Competitive_Score"].append(c)
    btn=soup.findAll("button", {"class":"ModeOption button-text"})
    values = []
    
# NUMBER OF HOUSES SOLD
    for i in btn:
        x = i.findAll('div',{"class":"tab"})
        for j in x:
            y=j.find('div',{"class":"value"})
            value = y.string.strip("%")
            values.append(value)
    stats["#of houses sold"].append(values[0])
    stats["Home sold above list price"].append(values[2])
    
# SCHOOL RATING
    risk = soup.find("div", {"class":"schools-content"})
    child = risk.findChildren("table",{"class":"basic-table-3"},recursive= True)
    ratings = []
    for x in child:
        for l in x.findAll('tr',{"class":"schools-table-row"}):
            for line in l.findAll('td',{"class":"rating-and-name-col"}):
                for y in line.findAll('div',{"class":"gs-rating-col"}):
                    rating =y.find('span',{"class":"rating-num"}).get_text()
                    ratings.append(int(rating))
        stats["School rating"].append(sum(ratings)/len(ratings))
    
# Merging into a dataframe

df = pd.DataFrame(stats)
df.index = ['Sunnyvale', 'San-Jose', 'Cupertino','Mountain-View','Milpitas','Palo-Alto','Santa Clara','Los Gatos','Campbell','Gilroy','Saratoga','Morgan-Hill',
           'Los-Altos']

# Writing to an excel

writer=pd.ExcelWriter('Cities statistics.xlsx')
df.to_excel(writer,'Cities statistics',index= True)
writer.save()


## Step 2: Selenium (City Level)

### Due to errros in using Selenium we have reverted to using Redfin's API consisting of all data points and readily available directly on their website: https://www.redfin.com/news/data-center/